# Demo Deep Belief Network para identificar TIPO de IRIS


0) Instalar paquete de DBN: 
( para más info ver https://github.com/albertbup/deep-belief-network/blob/master/README.md )

In [0]:
pip install git+git://github.com/albertbup/deep-belief-network.git

1) Cargar librerías:

In [0]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

from dbn.tensorflow import SupervisedDBNClassification
import numpy as np
import pandas as pd
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt

print ("Librerías cargadas.")

2) Configurar los datos cargados (se debe definir de acuerdo a los datos):

In [0]:
# define atributos y clases
CSV_COLUMN_NAMES = ['LargoSepalo', 'AnchoSepalo', 'LargoPetalo', 'AnchoPetalo', 'Clase']
CLASSES = ['Setosa', 'Versicolor', 'Virginica']

ClassAttributeName = 'Clase'

print("Configuración definida.")

3) Cargar CSV con datos a procesar y preparar datos para entrenar y probar (ya separados):

In [0]:
# levanta los datos de entrenamiento y prueba
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

# ver datos
train.head()
#test.head()

In [0]:
# define datos de entrada y salida para entrenamiento
X_train = np.array(train.drop([ClassAttributeName], axis=1))
Y_train = np.array(train[ClassAttributeName])

# define datos de entrada y salida para testing
X_test = np.array(test.drop([ClassAttributeName], axis=1))
Y_test = np.array(test[ClassAttributeName])

print("\n\nDatos Originales ", len(X_train)+len(X_test))
print("- Datos para Entrenar ", len(X_train))
print("- Datos para Probar ", len(X_test))

4) Especificar el modelo para la DBN:

In [0]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[512, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)

print(classifier)

5) Entrenar el modelo de la DBN:

In [0]:
classifier.fit(X_train, Y_train)

6) Evaluar el modelo de la RNA entrenado:

In [0]:
# ejecuta el  modelo
Y_pred = classifier.predict(X_test)

# calcula la exactitud
print('\nExactitud: %f' % accuracy_score(Y_test, Y_pred))

# muestra resultados de la predicción
classPreds = []
classReal = []
cantOK = 0
cantError = 0
print("\n Resultados: ")
for pred_class_id, expec in zip(Y_pred, Y_test):
    
    classPreds.append(CLASSES[pred_class_id])
    classReal.append(CLASSES[expec])

    if (pred_class_id == expec): 
      res = ""
      cantOK += 1
    else: 
      res = "!"
      cantError += 1

    print('Clase predecida es "{}"[{}], la correcta es "{}"[{}]{} '.format(
        CLASSES[pred_class_id], pred_class_id, CLASSES[expec], expec, res))

print('=== Total Ejemplos: {}, con {} predicciones ok y {} errores.'.format(cantOK+cantError, cantOK, cantError))

# gráfico de comparación
plt.title('Gráfico de Confusión')
plt.xlabel('Real')
plt.ylabel('DBN')
plt.scatter(classReal, classPreds)

# muestra reporte de clasificación
print("\n Reporte de Clasificación: ")
print(classification_report(classReal, classPreds))

# muestra matriz de confusion
print('\nMatriz de Confusión: ')
cm = confusion_matrix(classReal, classPreds, labels=CLASSES)
cmtx = pd.DataFrame(
    cm, 
    index=['r:{:}'.format(x) for x in CLASSES], 
    columns=['p:{:}'.format(x) for x in CLASSES]
  )
print(cmtx)
print("\n")


*) Hacer pruebas de predicciones:

In [0]:
# define ejemplos de prueba
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = [ [5.1, 3.3, 1.7, 0.5], [5.9, 3.0, 4.2, 1.5], [6.9, 3.1, 5.4, 2.1]]

print(predict_x)
print(expected)

In [0]:
# ejecuta el  modelo
predictions = classifier.predict(predict_x)

# muestra resultados de la predicción
for expec_class, pred in zip(expected, predictions):

  # muestra resultados
  print('Clase predecida es "{}", la correcta es "{}"'.format(
        CLASSES[pred],  expec_class))